In [2]:
import gdown
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

gdown.download('https://bit.ly/3GisL6J', 'ns_book4.csv', quiet=False)
ns_book4 = pd.read_csv('ns_book4.csv', low_memory=False)

def get_book_info(row):
  title = row['도서명']
  author = row['저자']
  pub = row['출판사']
  year = row['발행년도']
  url = 'http://www.yes24.com/product/search?query={}'
  r = requests.get(url.format(row['ISBN']))
  soup = BeautifulSoup(r.text, 'html.parser')

  try:
    if pd.isna(title):
      title = soup.find('a', attrs={'class': 'gd_name'}).get_text()
  except AttributeError:
    pass

  try:
    if pd.isna(author):
      authors = soup.find('span', attrs={'class': 'authPub info_auth'}).find_all('a').get_text()
      for i in authors:
        print(i)
        author.append(str(i))
      author = ','.join(list(author))
  except AttributeError:
    pass

  try:
    if pd.isna(pub):
      pub = soup.find('span', attrs={'class': 'authPub info_pub'}).find('a').get_text()
  except AttributeError:
    pass

  try:
    if pd.isna(year):
      year_str = soup.find('span', attrs={'class': 'authPub info_date'}).get_text()
      year = re.findall(r'\d{4}', year_str)
  except AttributeError:
    pass
  
  return title, author, pub, year

def data_fixing(ns_book4):
  ns_book4 = ns_book4.astype({'도서권수': 'int32', '대출건수': 'int32'})

  set_isbn_na_rows = ns_book4['세트 ISBN'].isna()
  ns_book4.loc[set_isbn_na_rows, '세트 ISBN'] = ''

  ns_book5 = ns_book4.replace({'발행년도': '.*(\d{4}).*'}, r'\1', regex=True)
  unknown_year = ns_book5['발행년도'].str.contains('\D', na=True)
  ns_book5.loc[unknown_year, '발행년도'] = '-1'

  ns_book5 = ns_book5.astype({'발행년도': 'int32'})

  dangun_yy_rows = ns_book5['발행년도'].gt(4000)
  ns_book5.loc[dangun_yy_rows, '발행년도'] = ns_book5.loc[dangun_yy_rows, '발행년도'] - 2333

  dangun_year = ns_book5['발행년도'].gt(4000)
  ns_book5.loc[dangun_year, '발행년도'] = -1

  old_books = ns_book5['발행년도'].gt(0) & ns_book5['발행년도'].lt(1900)
  ns_book5.loc[old_books, '발행년도'] = -1
  
  na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1)
  updated_sample = ns_book5[na_rows].apply(get_book_info, axis=1, result_type = 'expand')
  updated_sample.columns = ['도서명', '저자', '출판사', '발행년도']

  ns_book5.update(updated_sample)

  ns_book7 = ns_book5[(ns_book5['도서명'].notnull()) & (ns_book5['저자'].notnull()) & (ns_book5['출판사'].notnull()) & (ns_book5['발행년도'] != -1)]

  return ns_book7

data_fixing(ns_book4[70:80])

Downloading...
From: https://bit.ly/3GisL6J
To: /content/ns_book4.csv
100%|██████████| 55.5M/55.5M [00:00<00:00, 188MB/s]


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
70,71,처음 배우는 애저 (Azure Portal로 배우는 애저 도입부터 활용까지),김도균,한빛미디어,2020.0,9791162243695,,NaN,NaN,005.74,1,1,2021-03-15
71,72,질문이 답이 되는 순간 - 어떤 세상에서도 살아가야 할 우리에게 김제동과 전문가 7...,"김제동, 김상욱, 유현준, 심채경, 이원재, 정재승, 이정모, 김창남 (지은이)",나무의마음,2021.0,9791190457149,,NaN,NaN,001.3,1,0,2021-03-15
72,73,있는 그대로 - 정준영의 위빠사나 이야기,정준영 (지은이),에디터,2019.0,9788967442132,,NaN,NaN,224.81,1,1,2021-03-15
73,74,월 10만원 그림 투자 재테크 (주식보다 안전하고 부동산보다 수익 좋은),한혜미,쌤앤파커스,2021.0,9791165343170,,NaN,NaN,327.88,1,0,2021-03-15
74,75,우린 누군가의 봄이었으니까,이평,경향BP(경향비피),2019.0,9788969523334,,NaN,NaN,818,1,0,2021-03-15
75,76,우리는 히스테리 사회에 산다 (한국인의 사회적 성격 형성과 과정),강창동,박영스토리,2021.0,9791165191344,,NaN,NaN,331.1,1,1,2021-03-15
76,77,왕초보도 바로 돈 버는 부동산 경매의 기술 - 2천만 원으로 시작하는 실전 부동산 ...,"정민우, 유근용 (지은이)",비즈니스북스,2021.0,9791162541982,,NaN,NaN,367.564,1,0,2021-03-15
77,78,오늘의 주식 (이슈를 읽으면 종목이 보인다),효라클(김성효),토네이도,2021.0,9791158512064,,NaN,NaN,327.856,1,1,2021-03-15
78,79,아산 정주영 레거시,김화진,서울대학교출판문화원,2021.0,9788952129529,,NaN,NaN,325,1,1,2021-03-15
79,80,"쉽게 따라하는 인공지능 FOR 클래스룸 (인공지능 윤리, Google Arts & ...",박찬|전수연|진성임|손미현|노희진|정선재|강윤진|이정훈|변문경,다빈치books,2021.0,9791186742891,,NaN,NaN,373.33,1,1,2021-03-15
